# Preprocessing part with clustering algorithm in order to get clusters by RFMV values



### Among numerous metrics exists in retail and consumer goods industry, recency (R), frequency (F), and monetary (M) are often utilized in valuating customer. Marketers at times use variety (V) as well. Hence, we utilize all four metrics, abbreviated RFMV in our customer feature analysis.

In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import os


import scipy.sparse as sparse


%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install -U pyarrow  > /dev/null

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    AUX_DATA_ROOT = Path("/content/drive/My Drive/NEW_HACK_DATA")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.insert(0, str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from IPython.display import display, HTML

def print_df(df):
    display(HTML(df.to_html()))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
!pip install dask[dataframe] > /dev/null

import dask.dataframe as dd

# 1) Downloading data from the encoding part

In [ ]:
trans = pd.read_parquet(os.path.join(AUX_DATA_ROOT, 't.parquet'), engine='pyarrow', use_threads=True)
clients = pd.read_csv(os.path.join(AUX_DATA_ROOT, 'c.csv'))
materials = pd.read_csv(os.path.join(AUX_DATA_ROOT, 'm.csv'))
plants = pd.read_csv(os.path.join(AUX_DATA_ROOT, 'p.csv'))

In [ ]:
print(clients.shape)
print('birthyear unique values:', clients.birthyear.nunique())
print('city unique values:', clients.city.nunique())
print('ids unique values:', clients.client_id.nunique())

clients.describe()

clients.head()

(99995, 5)
birthyear unique values: 108
city unique values: 3
ids unique values: 99995


,Unnamed: 0,client_id,gender,city
count,99995.000000,99995.000000,99995.000000,99995.000000
mean,49997.000000,49998.959478,0.410721,1.079604
std,28866.214421,28867.703896,0.499131,0.426092
min,0.000000,0.000000,0.000000,0.000000
25%,24998.500000,24999.500000,0.000000,1.000000
50%,49997.000000,49999.000000,0.000000,1.000000
75%,74995.500000,74998.500000,1.000000,1.000000
max,99994.000000,99999.000000,2.000000,2.000000


,Unnamed: 0,client_id,gender,city,birthyear
0,0,10144,1,1,1990.0
1,1,61042,0,1,1969.0
2,2,83456,1,1,1976.0
3,3,7725,0,0,1966.0
4,4,12977,0,1,1988.0


In [ ]:
materials = materials.iloc[:,1:]
print(materials.shape)
print('vendor unique values:', materials.vendor.nunique())
print('hier_level_1 unique values:', materials.hier_level_1.nunique())
print('hier_level_2 unique values:', materials.hier_level_2.nunique())
print('hier_level_3 unique values:', materials.hier_level_3.nunique())
print('hier_level_4 unique values:', materials.hier_level_4.nunique())

print('vendors unique values for is_private_label==1:', materials.loc[materials.is_private_label==1, 'vendor'].nunique())

materials.head()

(105609, 8)
vendor unique values: 4185
hier_level_1 unique values: 2
hier_level_2 unique values: 60
hier_level_3 unique values: 391
hier_level_4 unique values: 2006
vendors unique values for is_private_label==1: 1096


,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,21972,1,56,250,1220,551,0,0
1,79310,1,56,250,1748,2519,0,0
2,99398,1,56,250,1070,2519,0,0
3,21754,1,56,291,580,2673,0,0
4,38468,1,56,291,580,2673,0,0


In [ ]:
plants = plants.iloc[:, 1:]
print(plants.shape)
print('city unique values:', plants.city.nunique())

plants.head()

(388, 4)
city unique values: 3


,Unnamed: 0,plant,plant_type,city
0,0,215,0,2
1,1,211,0,2
2,2,252,0,2
3,3,18,0,2
4,4,110,0,2


In [ ]:
print(trans.shape)
# print('min, max dates:', trans.chq_date.min(), trans.chq_date.max())
print('max length of bill:', trans.chq_position.nunique())
print('chq_id unique values:', trans.chq_id.nunique())

trans.describe()
trans.head()

(32109414, 9)
max length of bill: 459
chq_id unique values: 2964775


,chq_id,plant,client_id,material,sales_count,sales_sum,is_promo
count,3.210941e+07,3.210941e+07,3.210941e+07,3.210941e+07,3.210941e+07,3.210941e+07,3.210941e+07
mean,1.481767e+06,1.845276e+02,5.016135e+04,5.274385e+04,1.319230e+00,1.228870e+02,4.833838e-01
std,8.558720e+05,1.124506e+02,2.885262e+04,3.011254e+04,2.015387e+00,2.502002e+02,4.997238e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.656000e+03,-9.838800e+04,0.000000e+00
25%,7.406100e+05,8.400000e+01,2.500500e+04,2.691500e+04,1.000000e+00,3.999000e+01,0.000000e+00
50%,1.481309e+06,1.860000e+02,5.027400e+04,5.172000e+04,1.000000e+00,7.398000e+01,0.000000e+00
75%,2.223605e+06,2.790000e+02,7.513300e+04,7.768200e+04,1.000000e+00,1.399900e+02,1.000000e+00
max,2.964774e+06,3.870000e+02,9.999900e+04,1.056930e+05,3.000000e+03,1.985715e+05,1.000000e+00


,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,2390608,179,2016-11-01,004,56437,14213,2.0,146.98,0
1,2390608,179,2016-11-01,003,56437,7629,1.0,249.99,1
2,2390608,179,2016-11-01,012,56437,7264,1.0,47.99,1
3,2390608,179,2016-11-01,013,56437,7264,1.0,47.99,1
4,2390608,179,2016-11-01,011,56437,35784,2.0,53.98,1


### 2) Merging usefull tabels by keys

In [ ]:
# merge transactions with materials info

trans = pd.merge(trans, materials, on='material', how='left')
trans = dd.from_pandas(trans, npartitions=5)
trans.shape
trans.head()

(Delayed('int-24bd3d51-eca6-4309-aee4-2c81989625d2'), 16)

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,2390608,179,2016-11-01,004,56437,14213,2.0,146.98,0,0.0,32.0,130.0,936.0,2736.0,0.0,0.0
1,2390608,179,2016-11-01,003,56437,7629,1.0,249.99,1,0.0,10.0,305.0,1103.0,2278.0,0.0,0.0
2,2390608,179,2016-11-01,012,56437,7264,1.0,47.99,1,0.0,13.0,81.0,3.0,2715.0,0.0,1.0
3,2390608,179,2016-11-01,013,56437,7264,1.0,47.99,1,0.0,13.0,81.0,3.0,2715.0,0.0,1.0
4,2390608,179,2016-11-01,011,56437,35784,2.0,53.98,1,0.0,32.0,383.0,1775.0,1221.0,0.0,0.0


In [ ]:
trans['chq_date'] = dd.to_datetime(trans['chq_date'], format='%Y-%m-%d')

### 3) RFMV analysis

In [ ]:
import datetime as dt


# RECENCY
NOW = pd.to_datetime('2017-10-5', format='%Y-%m-%d') # just take the max date from the index + 1

def date_diff(df):
    return (NOW-df['Last Purchase']).dt.days

recency_df = trans.groupby('client_id')['chq_date'].max().reset_index()
recency_df.columns = ['Customer ID','Last Purchase']
recency_df['Recency'] = recency_df.map_partitions(date_diff)
recency_df = recency_df[['Customer ID', 'Recency']]

# recency_df.head()

# FREQUENCY
frequency_df = trans.groupby('client_id')['chq_date'].count().reset_index()
frequency_df.columns = ['Customer ID','Frequency']
# frequency_df.head()

# MONETARY
monetary_df = trans.groupby('client_id')['sales_sum'].sum().reset_index()
monetary_df.columns = ['Customer ID','Monetary']

# monetary_df.head()

# VARIETY
variety_df = trans.groupby('client_id')['material'].nunique().reset_index()
variety_df.columns = ['Customer ID','Variety']

# variety_df.head()

rfmv = recency_df.copy()
rfmv = rfmv.merge(frequency_df, on='Customer ID')
rfmv = rfmv.merge(monetary_df, on='Customer ID')
rfmv = rfmv.merge(variety_df, on='Customer ID')


In [ ]:
import pickle

with open('rfmv_quantiles.pickle', 'wb') as f:
    pickle.dump(rfmv_quantiles, f)

{'Frequency': {0.25: 80.0, 0.5: 189.0, 0.75: 418.0},
 'Monetary': {0.25: 9450.999999999996,
  0.5: 22556.999999999993,
  0.75: 50116.99999999998},
 'Recency': {0.25: 6.0, 0.5: 22.0, 0.75: 102.0},
 'Variety': {0.25: 65.0, 0.5: 141.0, 0.75: 276.0}}

In [ ]:
### we take the 25%, 50%, and 75% quantile for each of R, F, M, and V and store them into dictionary data type
rfmv_quantiles = rfmv.iloc[:, 1:].quantile(q = [0.25, 0.5, 0.75]).compute().to_dict()
rfmv_quantiles

In [ ]:
## create a function that returns a score--0 to 4 based on the quartile ranking
## these score can be easily interpreted by the K-Mean algorithm instead of the actual value of RFMB

### recency score: the lower the recency, the higher the score 
def RecencyScore(df, col, dict_):
    out = (df[col]<=dict_[col][0.25]).astype(int)*4 + \
          ((df[col]>dict_[col][0.25])&(df[col]<=dict_[col][0.5])).astype(int)*3 + \
          ((df[col]>dict_[col][0.5])&(df[col]<=dict_[col][0.75])).astype(int)*2 + \
          (df[col]>dict_[col][0.75]).astype(int)
    return out

    
# F, M, V: In contrast to Recency, the higher the quantile value, the higher the score    
def FMVScore(df, col, dict_):
    out = (df[col]<=dict_[col][0.25]).astype(int) + \
            ((df[col]>dict_[col][0.25])&(df[col]<=dict_[col][0.5])).astype(int)*2 + \
            ((df[col]>dict_[col][0.5])&(df[col]<=dict_[col][0.75])).astype(int)*3 + \
            (df[col]>dict_[col][0.75]).astype(int) * 4
    return out

rfmv2 = rfmv.copy()

rfmv2['R_q'] = rfmv2.map_partitions(RecencyScore, col='Recency', dict_=rfmv_quantiles)
rfmv2['F_q'] = rfmv2.map_partitions(FMVScore, col='Frequency', dict_=rfmv_quantiles)
rfmv2['M_q'] = rfmv2.map_partitions(FMVScore, col='Monetary', dict_=rfmv_quantiles)
rfmv2['V_q'] = rfmv2.map_partitions(FMVScore, col='Variety', dict_=rfmv_quantiles)

rfmv2 = rfmv2[['Customer ID', 'R_q', 'F_q', 'M_q', 'V_q',]]
rfmv2['Total_Score'] = rfmv2['R_q'] + rfmv2['F_q'] + rfmv2['M_q'] + rfmv2['V_q']

rfmv2 = rfmv2[['Customer ID', 'Total_Score']]

In [ ]:
import pickle

with open('clusters.pickle', 'wb') as f:
   pickle.dump(clusters, f)

### 4) Applying KMeans clustering algo

In [ ]:
# Apply KMean clustering using the optimal number with the "elbow". We just took a guess to use k = 4
kmeans = KMeans(n_clusters=4, 
                init='random', 
                random_state=None)

clusters = kmeans.fit_predict(rfmv2[['Total_Score']])

In [ ]:
rfmv_pandas = rfmv.compute()
# rfmv['Clusters'] = clusters

In [ ]:
rfmv_pandas['Clusters'] = clusters
path = Path("/content/drive/My Drive/Lenta_hack")
rfmv_pandas.to_parquet(os.path.join(path, 'rfmv.parquet'), index=False)

dask.dataframe.core.DataFrame

In [ ]:
rfmv.columns = ['client_id', 'Clusters']

trans = trans.merge(rfmv[['client_id','Clusters']], how='left', on='client_id')
trans = trans.merge(clients, how='left', on='client_id') # + gender, city

plants.colums = ['plant', 'plant_type', 'city_store_loc']
trans = trans.merge(plants, how='left', on='plant') # + plant_type, city

# New data with clusters and RFMV info you can find here:
### https://drive.google.com/file/d/1cwtTbZw6wrancDkB6raYHodrhiXUg3Ff/view?usp=sharing